In [0]:
par_tables = dbutils.widgets.get('Tables').split(',')
par_years = dbutils.widgets.get('Years').split(',')
par_filters = [filter.strip() for filter in dbutils.widgets.get('Filters').split(',')]

In [0]:
def move_files(source_path, target_path, file_filters):
    try:
        files = dbutils.fs.ls(source_path)
        for file in files:
            if all(filter in file.name for filter in file_filters) or len(file_filters) == 0:
                target_file_name = target_path + ('/' if target_path[-1] != '/' else '') + file.name
                dbutils.fs.mv(file.path, target_path + file.name)
                print(f'Moved {file.path.replace("dbfs:", "")} to {target_file_name}')
    except Exception as e:
        print(f'Error moving files: {str(e).split(":")[2]}')

In [0]:
def archive_files(table, year):
    source_path = f'/mnt/databricks/sleeper/stg/{table}/{year}'
    target_path = source_path.replace('stg', 'archive')

    move_files(source_path, target_path, par_filters)

def restore_files(table, year):
    source_path = f'/mnt/databricks/sleeper/archive/{table}/{year}'
    target_path = source_path.replace('archive', 'stg')

    move_files(source_path, target_path, par_filters)

for year in par_years:
    for table in par_tables:

        restore_files(table, year)


In [0]:
dbutils.fs.ls('/mnt/databricks/sleeper/archive/players/2024')

In [0]:
# for directory in dbutils.fs.ls('mnt/databricks/sleeper/archive/'):
#     files = dbutils.fs.ls(directory.path)

#     for file in files:
#         file_path = file.path.replace('dbfs:', '')
#         target_path = file_path.replace('2024', '2024/')
#         print(target_path)
#         dbutils.fs.mv(file_path, target_path)
#         print(f'Moved {file_path} to {target_path}')
        